### Universidad Autónoma de Aguascalientes

### Centro de Ciencias Básicas

### Departamento de Ciencias de la Computación

### Optativa Profesionalizante II: Machine Learning y Deep Learning

### 10° "A"

### Segunda Evaluación Parcial

### Docente: Dr. Francisco Javier Luna Rosas

### Alumno: Joel Alejandro Espinoza Sánchez (211800)

### Fecha de Entrega: Aguascalientes, Ags., 1 de mayo del 2023.

---

El análisis de sentimientos, a veces también denominado minería de opiniones, es una conocida sub-disciplina del amplio campo del PLN (Procesamiento del Lenguaje Natural); está relacionado con el análisis de la polaridad de documentos. Una tarea popular en el análisis de sentimiento es la clasificación de documentos basados en las emociones u opiniones expresadas de los autores respecto a un tema en particular. El conjunto de datos de críticas de cine consiste en 50,000 críticas de cine polarizadas etiquetadas como negativas y como positivas. Aquí, positiva significa que una película ha sido clasificada con más de seis estrellas, mientras que negativa significa que una película ha sido clasificada con menos de cinco estrellas.

El alumno deberá elaborar un documento (```*.pdf```) y un archivo auto-reproducible (```*.html```) que analice, implemente y evalúe algoritmos de Deep Learning y Machine Learning para clasificar las críticas de cine. El documento deberá contener:

- Portada

- Evidencias del examen

- Conclusiones

- Referencias (formato APA)

---

### a) Una explicación del preprocesamiento de datos para generar un formato adecuado de los datos

Primeramente se cargan los datos que se usarán:

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

data = pd.read_csv("movie_data.csv")

Se realiza un preprocesamiento con la librería NLTK para limpiar el texto, tokenizar, normalizar, eliminar las stopwords y pos_tagging con el siguiente código:

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tag import pos_tag
import re
nltk.download('averaged_perceptron_tagger')

def clean_text(text):
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Convertir a minúsculas
    text = text.lower()
    return text

def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

def normalize(tokens):
    # Stemming
    stemmer = SnowballStemmer('english')
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]
    return lemmatized_tokens

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if not token in stop_words]
    return filtered_tokens

def pos_tagging(tokens):
    pos_tokens = pos_tag(tokens)
    return pos_tokens

data['review'] = data['review'].apply(clean_text)
data['review'] = data['review'].apply(tokenize)
data['review'] = data['review'].apply(normalize)
data['review'] = data['review'].apply(remove_stopwords)
data['review'] = data['review'].apply(pos_tagging)

X = np.asarray(data['review'])
X = [' '.join([word[0] for word in sublist]) for sublist in X]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\alexe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### b) Una explicación del modelo bolsa de palabras o cualquier otro analizador ligústico aplicado al Dataset de críticas de cine

El modelo de bolsa de palabras que se usó fue dado por TensorFlow de su apartado de Keras siendo el método ```Tokenizer``` de la siguiente forma:

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)

### c) Una explicación de la transformación de las palabras en vectores de características (utilice la frecuencia de termino - frecuencia inversa de documento “tf-idf” o cualquier otra técnica que permita transformar palabras a vectores de características)

La transformación de palabras en vectores de características se usa nuevamente TensorFlow con el uso de Keras usando ```pad_sequences``` de la siguiente forma:

In [7]:
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=200)

### d) Una explicación del modelo CNN (Convoluciones 1D) para clasificar las críticas de cine. La precisión del modelo debe ser del 95% o mayor

Explicación

In [16]:
y = np.array(data['sentiment'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

model = Sequential()
model.add(Embedding(5000, 32, input_length=200))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/5
1250/1250 [==============================] - 13s 10ms/step - loss: 0.4028 - accuracy: 0.8170 - val_loss: 0.3212 - val_accuracy: 0.8603
Epoch 2/5
1250/1250 [==============================] - 10s 8ms/step - loss: 0.2602 - accuracy: 0.8928 - val_loss: 0.3084 - val_accuracy: 0.8675
Epoch 3/5
1250/1250 [==============================] - 11s 9ms/step - loss: 0.1993 - accuracy: 0.9239 - val_loss: 0.3211 - val_accuracy: 0.8687
Epoch 4/5
1250/1250 [==============================] - 13s 11ms/step - loss: 0.1436 - accuracy: 0.9495 - val_loss: 0.3343 - val_accuracy: 0.8672
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.3796 - accuracy: 0.8658
Loss:  0.3795654773712158
Accuracy:  0.8658000230789185


### e) Una explicación del análisis comparativo de modelos de Deep Learning (Redes Neuronales, Redes CNN (Convoluciones 1D)) Vs Machine Learning (KNN, Bayes, Arboles de Decisión, SVM, Random Forest, Potenciación, etc.), compare al menos dos clasificadores de cada uno con una precisión del 95% o mayor, el análisis deberá comparar: la precisión del modelo, el error del modelo, precisión negativa (especificidad), precisión positiva (sensibilidad), falsos positivos, falsos negativos, asertividad positiva, asertividad negativa

a

In [10]:
print('a')

a


---

## Conclusiones

Es interesante e importante poder implementar las bases de estos temas para entenderlos en un futuro, pues, posteriormente no basta con sólo importar librerías que realicen el trabajo pesado, ya que, implementar manualmente estos algoritmos nos enseña a qué hay detrás del algoritmo, cómo funciona y poder comprender realmente qué está ocurriendo como la base de una red neuronal convolucional y la forma en la que ésta aprende. Es muy útil la implementación de estos algoritmos en estas tareas para las futuras tareas de la materia y aplicaciones de Machine Learning en la vida personal.

## Referencias

- Anónimo (s.f.) “Red neuronal artificial”. Obtenido de Wikipedia: https://es.wikipedia.org/wiki/Red_neuronal_artificial.
- Data Scientest (2021) “Perceptrón. ¿Qué es y para qué sirve?”. Obtenido de Data Scientest: https://datascientest.com/es/perceptron-que-es-y-para-que-sirve.
- Luna, F. (2023) “El Modelo de McCulloch – Pitts”. Apuntes de ICI 10°.